<a href="https://colab.research.google.com/github/waevd/Mid_Term_2017732021_-/blob/main/2017732021_%EC%9D%B4%EC%84%B1%EB%AF%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate'https://docs.google.com/uc?export=download&id=1XF1-KJLVSS-NTUxy_t-4qvYGsAQMaQId' -O- | sed -rn's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1XF1-KJLVSS-NTUxy_t-4qvYGsAQMaQId" -O hand_count_dataset.zip && rm -rf ~/cookies.txt

sed: invalid option -- '/'
Usage: sed [OPTION]... {script-only-if-no-other-script} [input-file]...

  -n, --quiet, --silent
                 suppress automatic printing of pattern space
      --debug
                 annotate program execution
  -e script, --expression=script
                 add the script to the commands to be executed
  -f script-file, --file=script-file
                 add the contents of script-file to the commands to be executed
  --follow-symlinks
                 follow symlinks when processing in place
  -i[SUFFIX], --in-place[=SUFFIX]
                 edit files in place (makes backup if SUFFIX supplied)
  -l N, --line-length=N
                 specify the desired line-wrap length for the `l' command
  --posix
                 disable all GNU extensions.
  -E, -r, --regexp-extended
                 use extended regular expressions in the script
                 (for portability use POSIX -E).
  -s, --separate
                 consider files as separate rathe

In [3]:
!unzip hand_count_dataset.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: hand_count_dataset/train_dataset/1/2821.JPG  
  inflating: hand_count_dataset/train_dataset/1/2822.JPG  
  inflating: hand_count_dataset/train_dataset/1/2823.JPG  
  inflating: hand_count_dataset/train_dataset/1/2824.JPG  
  inflating: hand_count_dataset/train_dataset/1/2825.JPG  
  inflating: hand_count_dataset/train_dataset/1/2826.JPG  
  inflating: hand_count_dataset/train_dataset/1/2827.JPG  
  inflating: hand_count_dataset/train_dataset/1/2828.JPG  
  inflating: hand_count_dataset/train_dataset/1/2829.JPG  
  inflating: hand_count_dataset/train_dataset/1/283.JPG  
  inflating: hand_count_dataset/train_dataset/1/2830.JPG  
  inflating: hand_count_dataset/train_dataset/1/2831.JPG  
  inflating: hand_count_dataset/train_dataset/1/2832.JPG  
  inflating: hand_count_dataset/train_dataset/1/2833.JPG  
  inflating: hand_count_dataset/train_dataset/1/2834.JPG  
  inflating: hand_count_dataset/train_dataset/1/2835.JPG  
  inflating: hand_cou

In [13]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
from torch.utils.data import DataLoader

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)


# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)


class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer4 = torch.nn.Sequential(
            torch.nn.Linear(128*28*28,625),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5))

        self.fc = torch.nn.Linear(625,10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
      out = self.layer1(x)
      out = self.layer2(out)
      out = self.layer3(out)
      out = out.view(out.size(0), -1)
      out = self.layer4(out)
      out = self.fc(out)
      return out

# parameters
##### 적당히 수정한다
learning_rate = 0.001
training_epochs = 10

# contruct model
model = CNN().to(device)

# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# train
##### 직접 dataset을 만들어 학습할 때 자주 사용되는 함수이니 기능을 숙지한다
##### pytorch 공식 홈페이지에서 관련 함수의 정보를 직접 찾아보는 것을 추천한다
trans = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
 ])

train_data = dsets.ImageFolder(root='/content/hand_count_dataset/train_dataset', transform=trans)
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True)

total_batch = len(data_loader)

##### 학습이 이루어지는 loop
##### 지난 source code를 그대로 사용해도 되지만, 연습삼아 직접 작성해보자

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

    trans=transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor()])
    valid_data = dsets.ImageFolder(root='/content/hand_count_dataset/valid_dataset', transform=trans)
    valid_set = DataLoader(dataset = valid_data, batch_size = len(valid_data))


    with torch.no_grad():

      for X, Y in valid_set:
          X = X.to(device)
          Y = Y.to(device)

          prediction = model(X)
          correct_prediction = torch.argmax(prediction, 1) == Y

      accuracy = correct_prediction.float().mean()


      print('Accuracy:', accuracy.item())

print('Learning Finished! Test start')

trans=transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor()])
test_data = dsets.ImageFolder(root='/content/hand_count_dataset/test_dataset', transform=trans)
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))


with torch.no_grad():

    for X, Y in test_set:
        X = X.to(device)
        Y = Y.to(device)

        prediction = model(X)
        correct_prediction = torch.argmax(prediction, 1) == Y
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

cuda
[Epoch:    1] cost = 0.561321437
Accuracy: 0.6266666650772095
[Epoch:    2] cost = 0.144103602
Accuracy: 0.753333330154419
[Epoch:    3] cost = 0.0694893524
Accuracy: 0.8400000333786011
[Epoch:    4] cost = 0.0370312221
Accuracy: 0.8866667151451111
[Epoch:    5] cost = 0.0303445514
Accuracy: 0.8966667056083679
[Epoch:    6] cost = 0.0313085727
Accuracy: 0.9066666960716248
[Epoch:    7] cost = 0.0223943852
Accuracy: 0.9266666769981384
[Epoch:    8] cost = 0.0162154529
Accuracy: 0.9266666769981384
[Epoch:    9] cost = 0.0194982793
Accuracy: 0.9166666865348816
[Epoch:   10] cost = 0.0132956784
Accuracy: 0.8600000143051147
Learning Finished! Test start
Accuracy: 0.5366666913032532


In [ ]:
!git clone "https://github.com/waevd/Mid_Term_2017732021_-/tree/main"
